In [46]:
from src.gdrive_api.folder_upload import upload_folder
from utils import service_account_path



MAIN_DIR = 'https://drive.google.com/drive/folders/1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'



In [1]:
import concurrent.futures
import io
import json
import os
import sys
import threading
import traceback

import nbformat
from dotenv import find_dotenv, load_dotenv
from fuzzywuzzy import fuzz
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from openai import OpenAI
from utils import get_delivered_df, DATA_DIR, PROJECT_ROOT

from src.llm_reviewer.constants import Roles
from src.llm_reviewer.llm_api import LLMAPIFactory, make_llm_request


def download_file(service_account_file, file_id, revision_id=None):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file, scopes=["https://www.googleapis.com/auth/drive"]
    )
    service = build("drive", "v3", credentials=credentials)

    # Request to download the file, optionally specifying a revision
    if revision_id is not None:
        request = service.revisions().get_media(fileId=file_id, revisionId=revision_id)
    else:
        request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    # Download the file
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download progress: %d%%." % int(status.progress() * 100))

    # Move the buffer's pointer to the beginning
    fh.seek(0)

    # Read the notebook content as JSON
    payload = json.load(fh)
    return payload


/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
import utils
from utils import service_account_path
from src.sheets_utils import download_sheet_as_df

sheet_id = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'
redo_df = download_sheet_as_df(service_account_path, sheet_id, 'gpt_flags_2')

In [3]:
len(redo_df)

59

45

In [4]:
from utils import get_delivered_df

batch_5 = get_delivered_df([5])

In [8]:
done_df = done_df.assign(file_id=done_df['task_link'].apply(lambda x: x.split('/')[-1]))


In [5]:
batch_5 = batch_5.assign(jsonl_file_id=batch_5['jsonl_link'].apply(lambda x: x.split('/file/d/')[1].split('/')[0]))
batch_5

,task_link,jsonl_link,number_of_turns,batch_id,jsonl_file_id
0,https://colab.research.google.com/drive/1IMCwR...,https://drive.google.com/file/d/10tqk4wcBJFbzb...,2,5,10tqk4wcBJFbzb1SgvVn8U2By7Uenjz62
1,https://colab.research.google.com/drive/1d613I...,https://drive.google.com/file/d/135lxPXEWHb2Pi...,2,5,135lxPXEWHb2Piqy6kdRuxsp99DMvOJ4f
2,https://colab.research.google.com/drive/1qJPPq...,https://drive.google.com/file/d/17faYUO_xOnLan...,1,5,17faYUO_xOnLanEmPfGx5zNVnW0MXVhaV
3,https://colab.research.google.com/drive/1cP6qz...,https://drive.google.com/file/d/12lm8x2QikaLaZ...,1,5,12lm8x2QikaLaZbF6S2PLdKcbyEGwbV-O
4,https://colab.research.google.com/drive/1xw5RA...,https://drive.google.com/file/d/1t9D3joY980t40...,3,5,1t9D3joY980t40zbvBxEDo2emtIuBaH3k
...,...,...,...,...,...
1055,https://colab.research.google.com/drive/1QXk9F...,https://drive.google.com/file/d/1S4dpvl7lDK0Ix...,2,5,1S4dpvl7lDK0IxkRN3tq4xPnGMd30Sr6V
1056,https://colab.research.google.com/drive/1lekWL...,https://drive.google.com/file/d/1BPxrto_U7ZCiy...,2,5,1BPxrto_U7ZCiyBdkVq5lzfd6E4lLpxeP
1057,https://colab.research.google.com/drive/1s_KMO...,https://drive.google.com/file/d/1rBeAk3HCK_aVG...,1,5,1rBeAk3HCK_aVGXluG13K6ejF-WjHZxbn
1058,https://colab.research.google.com/drive/1PFqmH...,https://drive.google.com/file/d/15de0-zkqjJxqK...,7,5,15de0-zkqjJxqKONeRcUrTIp0KJtWho8v


In [6]:
done_df = redo_df[redo_df['status'] == 'Done']
done_df = done_df.merge(batch_5[['task_link', 'jsonl_file_id']], on='task_link', how='left')
print(len(done_df))
done_df


47


,task_link,avg_score,min_score,issues,original_author_email,resolved_by_email,status,duration_mins,comments,jsonl_file_id
0,https://colab.research.google.com/drive/1uc9gj...,3.833333,3,Overall - Completness: \n - Assistant's initia...,toh.y@turing.com,zain.v@turing.com,Done,10,None,1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur
1,https://colab.research.google.com/drive/1A4jBO...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,toh.y@turing.com,zain.v@turing.com,Done,15,None,1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H
2,https://colab.research.google.com/drive/18C1mY...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,20,None,1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv
3,https://colab.research.google.com/drive/1_Iypd...,3.833333,3,Overall - Completness: \n - The conversation l...,gautam.g@turing.com,khalid.s@turing.com,Done,45,None,17zNEPanui4_3NjvFe4184S0nNKRtt4HD
4,https://colab.research.google.com/drive/1uA3D1...,3.833333,3,Overall - Completness: \n - Assistant's last r...,adil.m@turing.com,adil.m@turing.com,Done,30,None,1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH
5,https://colab.research.google.com/drive/1RqZj4...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,14,None,1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8
6,https://colab.research.google.com/drive/17LOxl...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,caram.v@turing.com,caram.v@turing.com,Done,30,None,1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb
7,https://colab.research.google.com/drive/1PGIEJ...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,ishwar.b@turing.com,ishwar.b@turing.com,Done,45,None,1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg
8,https://colab.research.google.com/drive/141lAs...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,andranik.g@turing.com,safi.u@turing.com,Done,25,None,1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc
9,https://colab.research.google.com/drive/1Vr2BH...,3.833333,3,Overall - Completness: \n - The assistant's re...,pawan.s@turing.com,zain.v@turing.com,Done,12,None,1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5


In [7]:
file_id = done_df.iloc[0]['jsonl_file_id']
download_file(service_account_path, file_id)

Download progress: 100%.


{'metadata': {'task_link': 'https://colab.research.google.com/drive/1uc9gjNVbLismCaXtOYZJLOGidNH-EBJY',
  'number_of_turns': '4',
  'use_case__summary': 'User seeks assistance for coding task to merge lists into a dictionary.',
  'behavioral_tags': [{'top_level': 'continuation follow up',
    'sub_level': 'request for clarification/elaboration',
    'custom_category': 'FALSE'},
   {'top_level': 'respond to assistant',
    'sub_level': 'clarify an ambiguity',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'supplement/extend',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'request alternatives',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'incrementally build',
    'custom_category': 'FALSE'}],
  'programming_language_tags': [{'language': 'python', 'percentage': '1'}],
  'dependency_tags': [{'dependency': 'itertools', 'percentage': '0.2'}],
  'topi

In [8]:
from jsonl_dump import *



def download_parse_delivered_into_jsonl(
    batch_df, max_workers=20, no_work=False
):
    delivered_df = batch_df  # Assuming batch_id 4 is required
    if not no_work:
        parsed_conversations = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            file_ids = [link.split("/")[-1] for link in delivered_df["task_link"]]
            parsed_conversations = list(
                executor.map(
                    download_and_parse_notebook,
                    [service_account_path] * len(file_ids),
                    file_ids,
                )
            )

        for conversation in parsed_conversations:
            if conversation is None:
                continue
            colab_link = conversation["colab_link"]
            batch_id = None
            for b_id, task_link in zip(
                delivered_df["batch_id"], delivered_df["task_link"]
            ):
                if colab_link.endswith(task_link.split("/")[-1]):
                    batch_id = b_id
                    break
            if batch_id is None:
                raise
            batch_name = f"batch_{batch_id}"
            batch_folder = f"{DATA_DIR}jsonl_conversations/{batch_name}/"
            if not os.path.exists(batch_folder):
                os.makedirs(batch_folder)
            drive_id = conversation["id"]
            with open(f"{batch_folder}{drive_id}.json", "w") as f:
                f.write(json.dumps({"batch_id": batch_id, **conversation}))
    else:
        parsed_conversations = []
        for batch_id in batch_ids:
            batch_name = f"batch_{batch_id}"
            batch_folder = f"{DATA_DIR}jsonl_conversations/{batch_name}/"
            if os.path.exists(batch_folder):
                for file_name in os.listdir(batch_folder):
                    if file_name.endswith(".json"):
                        with open(f"{batch_folder}{file_name}", "r") as f:
                            conversation = json.load(f)
                            parsed_conversations.append(conversation)
            else:
                raise Exception(f"Batch folder for {batch_name} not found")

    return {"delivered_df": delivered_df, "conversations": parsed_conversations}

batch_5['batch_id'] = 66

batch_df = batch_5[batch_5['task_link'].isin(done_df['task_link'])]
data = download_parse_delivered_into_jsonl(batch_df)

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing 

In [ ]:
data.

In [11]:
len(data['conversations'])

47

In [12]:
convos = data['conversations']
convos[0]

{'id': '1dZpsB4RVjM2gkv17yULuE4kTQ7bVS9oh',
 'colab_link': 'https://colab.research.google.com/drive/1dZpsB4RVjM2gkv17yULuE4kTQ7bVS9oh',
 'revision_id': None,
 'metadata': {'topic': 'python_language_and_scripting > advanced_networking',
  'type': 'modification',
  'target_turns': '1'},
 'messages': [{'role': 'User',
   'content': "Hey, I've got this snippet that uses the socket library to create a simple server, but I want to modify it to handle multiple clients using threading. Can you help me with that? Here's the code:\n```python\nimport socket\n\ns = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\ns.bind(('localhost', 12345))\ns.listen(5)\nwhile True:\n    clientsocket, address = s.accept()\n    print(f'Connection from {address} has been established!')\n    clientsocket.send(bytes('Welcome to the server!', 'utf-8'))\n    clientsocket.close()\n```",
   'type': 'markdown'},
  {'role': 'Assistant',
   'content': "Yes, I can! You can handle multiple clients using threading. In this c

In [13]:
import pandas as pd

# Create a DataFrame with only 'colab_link' and 'messages' from the conversations list
convos_df = pd.DataFrame(convos, columns=['colab_link', 'messages'])
convos_df

,colab_link,messages
0,https://colab.research.google.com/drive/1dZpsB...,"[{'role': 'User', 'content': 'Hey, I've got th..."
1,https://colab.research.google.com/drive/15vi3d...,"[{'role': 'User', 'content': 'I need to add a ..."
2,https://colab.research.google.com/drive/17jSti...,"[{'role': 'User', 'content': 'Please update my..."
3,https://colab.research.google.com/drive/1jC380...,"[{'role': 'User', 'content': 'Can you modify t..."
4,https://colab.research.google.com/drive/1jLQa0...,"[{'role': 'User', 'content': 'I'm trying to en..."
5,https://colab.research.google.com/drive/1MHWMj...,"[{'role': 'User', 'content': 'I want to modify..."
6,https://colab.research.google.com/drive/17NsdA...,"[{'role': 'User', 'content': 'How can I shuffl..."
7,https://colab.research.google.com/drive/1itp5t...,"[{'role': 'User', 'content': 'I'm preparing fo..."
8,https://colab.research.google.com/drive/1uA3D1...,"[{'role': 'User', 'content': 'Hi I want to pre..."
9,https://colab.research.google.com/drive/1CpbJq...,"[{'role': 'User', 'content': 'I am preparing f..."


In [15]:
merged_df = pd.merge(done_df, convos_df, left_on='task_link', right_on='colab_link')
merged_df


,task_link,avg_score,min_score,issues,original_author_email,resolved_by_email,status,duration_mins,comments,jsonl_file_id,colab_link,messages
0,https://colab.research.google.com/drive/1uc9gj...,3.833333,3,Overall - Completness: \n - Assistant's initia...,toh.y@turing.com,zain.v@turing.com,Done,10,None,1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur,https://colab.research.google.com/drive/1uc9gj...,"[{'role': 'User', 'content': 'I'm trying to co..."
1,https://colab.research.google.com/drive/1A4jBO...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,toh.y@turing.com,zain.v@turing.com,Done,15,None,1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H,https://colab.research.google.com/drive/1A4jBO...,"[{'role': 'User', 'content': '```bash Tracebac..."
2,https://colab.research.google.com/drive/18C1mY...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,20,None,1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv,https://colab.research.google.com/drive/18C1mY...,"[{'role': 'User', 'content': 'Hello, I am runn..."
3,https://colab.research.google.com/drive/1_Iypd...,3.833333,3,Overall - Completness: \n - The conversation l...,gautam.g@turing.com,khalid.s@turing.com,Done,45,None,17zNEPanui4_3NjvFe4184S0nNKRtt4HD,https://colab.research.google.com/drive/1_Iypd...,"[{'role': 'User', 'content': 'Could you demons..."
4,https://colab.research.google.com/drive/1uA3D1...,3.833333,3,Overall - Completness: \n - Assistant's last r...,adil.m@turing.com,adil.m@turing.com,Done,30,None,1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH,https://colab.research.google.com/drive/1uA3D1...,"[{'role': 'User', 'content': 'Hi I want to pre..."
5,https://colab.research.google.com/drive/1RqZj4...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,14,None,1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8,https://colab.research.google.com/drive/1RqZj4...,"[{'role': 'User', 'content': 'I'm working on a..."
6,https://colab.research.google.com/drive/17LOxl...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,caram.v@turing.com,caram.v@turing.com,Done,30,None,1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb,https://colab.research.google.com/drive/17LOxl...,"[{'role': 'User', 'content': 'I have an e-comm..."
7,https://colab.research.google.com/drive/1PGIEJ...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,ishwar.b@turing.com,ishwar.b@turing.com,Done,45,None,1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg,https://colab.research.google.com/drive/1PGIEJ...,"[{'role': 'User', 'content': 'Hey, I am prepar..."
8,https://colab.research.google.com/drive/141lAs...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,andranik.g@turing.com,safi.u@turing.com,Done,25,None,1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc,https://colab.research.google.com/drive/141lAs...,"[{'role': 'User', 'content': 'I am working wit..."
9,https://colab.research.google.com/drive/1Vr2BH...,3.833333,3,Overall - Completness: \n - The assistant's re...,pawan.s@turing.com,zain.v@turing.com,Done,12,None,1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5,https://colab.research.google.com/drive/1Vr2BH...,"[{'role': 'User', 'content': 'Hi, I am getting..."


In [43]:
jsonl_files = []
for index, row in merged_df.iterrows():
    jsonl_files.append(download_file(service_account_path, row['jsonl_file_id']))




Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


In [44]:
for index, row in merged_df.iterrows():
    # Find the corresponding item in jsonl_files where task_link matches the current row's task_link
    matching_file = next((item for item in jsonl_files if item['metadata']['task_link'] == row['task_link']), None)
    if matching_file:
        print(row['messages'] == matching_file['messages'])
    else:
        raise


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
True
False
False
True
False
False
True
True
True
True
True


In [17]:

merged_df['jsonl_raw'] = merged_df['jsonl_file_id'].apply(lambda x: download_file(service_account_path, x))

merged_df


Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


,task_link,avg_score,min_score,issues,original_author_email,resolved_by_email,status,duration_mins,comments,jsonl_file_id,colab_link,messages,jsonl_raw
0,https://colab.research.google.com/drive/1uc9gj...,3.833333,3,Overall - Completness: \n - Assistant's initia...,toh.y@turing.com,zain.v@turing.com,Done,10,None,1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur,https://colab.research.google.com/drive/1uc9gj...,"[{'role': 'User', 'content': 'I'm trying to co...",{'metadata': {'task_link': 'https://colab.rese...
1,https://colab.research.google.com/drive/1A4jBO...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,toh.y@turing.com,zain.v@turing.com,Done,15,None,1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H,https://colab.research.google.com/drive/1A4jBO...,"[{'role': 'User', 'content': '```bash Tracebac...",{'metadata': {'task_link': 'https://colab.rese...
2,https://colab.research.google.com/drive/18C1mY...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,20,None,1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv,https://colab.research.google.com/drive/18C1mY...,"[{'role': 'User', 'content': 'Hello, I am runn...",{'metadata': {'task_link': 'https://colab.rese...
3,https://colab.research.google.com/drive/1_Iypd...,3.833333,3,Overall - Completness: \n - The conversation l...,gautam.g@turing.com,khalid.s@turing.com,Done,45,None,17zNEPanui4_3NjvFe4184S0nNKRtt4HD,https://colab.research.google.com/drive/1_Iypd...,"[{'role': 'User', 'content': 'Could you demons...",{'metadata': {'task_link': 'https://colab.rese...
4,https://colab.research.google.com/drive/1uA3D1...,3.833333,3,Overall - Completness: \n - Assistant's last r...,adil.m@turing.com,adil.m@turing.com,Done,30,None,1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH,https://colab.research.google.com/drive/1uA3D1...,"[{'role': 'User', 'content': 'Hi I want to pre...",{'metadata': {'task_link': 'https://colab.rese...
5,https://colab.research.google.com/drive/1RqZj4...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,14,None,1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8,https://colab.research.google.com/drive/1RqZj4...,"[{'role': 'User', 'content': 'I'm working on a...",{'metadata': {'task_link': 'https://colab.rese...
6,https://colab.research.google.com/drive/17LOxl...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,caram.v@turing.com,caram.v@turing.com,Done,30,None,1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb,https://colab.research.google.com/drive/17LOxl...,"[{'role': 'User', 'content': 'I have an e-comm...",{'metadata': {'task_link': 'https://colab.rese...
7,https://colab.research.google.com/drive/1PGIEJ...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,ishwar.b@turing.com,ishwar.b@turing.com,Done,45,None,1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg,https://colab.research.google.com/drive/1PGIEJ...,"[{'role': 'User', 'content': 'Hey, I am prepar...",{'metadata': {'task_link': 'https://colab.rese...
8,https://colab.research.google.com/drive/141lAs...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,andranik.g@turing.com,safi.u@turing.com,Done,25,None,1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc,https://colab.research.google.com/drive/141lAs...,"[{'role': 'User', 'content': 'I am working wit...",{'metadata': {'task_link': 'https://colab.rese...
9,https://colab.research.google.com/drive/1Vr2BH...,3.833333,3,Overall - Completness: \n - The assistant's re...,pawan.s@turing.com,zain.v@turing.com,Done,12,None,1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5,https://colab.research.google.com/drive/1Vr2BH...,"[{'role': 'User', 'content': 'Hi, I am getting...",{'metadata': {'task_link': 'https://colab.rese...


In [40]:
rj_last = download_file(service_account_path, merged_df.iloc[-1]['jsonl_file_id'])

Download progress: 100%.


In [20]:
j1 = dict(merged_df.iloc[0]['jsonl_raw'])
j2 = dict(merged_df.iloc[0]['jsonl_raw'])

In [22]:
j11 = dict(merged_df.iloc[0]['jsonl_raw'])
j22 = dict(merged_df.iloc[0]['jsonl_raw'])

In [39]:
rj3['messages'] == merged_df['messages'].iloc[2]

True

In [42]:
rj_last['messages'] == merged_df['messages'].iloc[-1]

True

In [26]:
j11['messages']

[{'role': 'User',
  'content': "I'm trying to combine two lists into a dictionary, where one list contains the keys and the other contains the values. However, the lists are not the same length. Can you modify my code to fill in missing values with `None`? Here are my lists: `keys = ['a', 'b', 'c']` and `values = [1, 2]`.",
  'type': 'markdown'},
 {'role': 'Assistant',
  'content': "Hey, have you had the opportunity to utilize the `zip_longest` function from the `itertools` module? It's quite useful when you need to merge two lists into a dictionary and ensure there are no missing values, as it conveniently fills the gaps with `None`.\n\nCheck out how it works in this example:",
  'type': 'markdown'},
 {'role': 'Assistant',
  'content': "from itertools import zip_longest\n\nkeys = ['a', 'b', 'c']\nvalues = [1, 2]\n\n# Combine lists into a dictionary with missing values filled with None\nresult_dict = dict(zip_longest(keys, values, fillvalue=None))\n\nprint(result_dict)",
  'type': 'cod

In [21]:
for index, row in merged_df.iterrows():
    if isinstance(row['jsonl_raw'], dict):
        row['jsonl_raw']['messages'] = row['messages']


In [47]:
len(merged_df)

47

In [49]:
merged_df['jsonl_raw'].iloc[1]

{'metadata': {'task_link': 'https://colab.research.google.com/drive/1A4jBOVVRiVjXRrSc6Khn5_nQKRj1wbmw',
  'number_of_turns': '5',
  'use_case__summary': 'User employs Assistant to resolve programming issues and manage software versions.',
  'behavioral_tags': [{'top_level': 'continuation follow up',
    'sub_level': 'request for clarification/elaboration',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'incrementally build',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'supplement/extend',
    'custom_category': 'FALSE'},
   {'top_level': 'respond to assistant',
    'sub_level': 'answer a question',
    'custom_category': 'FALSE'},
   {'top_level': 'user actions',
    'sub_level': 'user requests alternative solution',
    'custom_category': 'TRUE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'request alternatives',
    'custom_category': 'FALSE'},
   {'top_level': 'respond to 

In [54]:
merged_df.columns

Index(['task_link', 'avg_score', 'min_score', 'issues',
       'original_author_email', 'resolved_by_email', 'status', 'duration_mins',
       'comments', 'jsonl_file_id', 'colab_link', 'messages', 'jsonl_raw'],
      dtype='object')

In [48]:
merged_df['jsonl_raw'].iloc[0]

{'metadata': {'task_link': 'https://colab.research.google.com/drive/1uc9gjNVbLismCaXtOYZJLOGidNH-EBJY',
  'number_of_turns': '4',
  'use_case__summary': 'User seeks assistance for coding task to merge lists into a dictionary.',
  'behavioral_tags': [{'top_level': 'continuation follow up',
    'sub_level': 'request for clarification/elaboration',
    'custom_category': 'FALSE'},
   {'top_level': 'respond to assistant',
    'sub_level': 'clarify an ambiguity',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'supplement/extend',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'request alternatives',
    'custom_category': 'FALSE'},
   {'top_level': 'continuation follow up',
    'sub_level': 'incrementally build',
    'custom_category': 'FALSE'}],
  'programming_language_tags': [{'language': 'python', 'percentage': '1'}],
  'dependency_tags': [{'dependency': 'itertools', 'percentage': '0.2'}],
  'topi

In [53]:
merged_df[['task_link', 'jsonl_link', 'jsonl_file_id']]

KeyError: "['jsonl_link'] not in index"

In [ ]:
def get_file_id(service: Resource, file_name: str, parent_id: str) -> Optional[str]:
    """Retrieve the ID of a file in Google Drive.

    Args:
        service: The Google Drive service resource.
        file_name: The name of the file to find.
        parent_id: The ID of the parent folder.

    Returns:
        The ID of the file or None if not found.
    """
    print("Getting file id...")
    query = f"name = '{file_name}' and '{parent_id}' in parents and trashed = false"
    response = (
        service.files()
        .list(q=query, spaces="drive", fields="files(id, name)")
        .execute()
    )
    print("processing response...")
    for file in response.get("files", []):
        if file.get("name") == file_name:
            return file.get("id")
    return None

In [60]:
file_name = os.path.basename(jsonl_filename)
file_metadata = {"name": file_name, "parents": [MAIN_DIR]}

In [61]:
file_metadata

{'name': '15XEH5ZQp2bc9d1uLlPdiQ79mwhTceVHn.json',
 'parents': ['https://drive.google.com/drive/folders/1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR']}

In [58]:
from src.gdrive_api.utils import get_file_id

get_file_id(build_service(service_account_path), jsonl_filename, MAIN_DIR)

Getting file id...


HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files?q=name+%3D+%27%2Fhome%2Fsaya%2Fchario%2Fupstream_character_tasks%2Fdata%2F1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json%27+and+%27https%3A%2F%2Fdrive.google.com%2Fdrive%2Ffolders%2F1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR%27+in+parents+and+trashed+%3D+false&spaces=drive&fields=files%28id%2C+name%29&alt=json returned "File not found: .". Details: "[{'message': 'File not found: .', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">

In [64]:
MAIN_DIR

'https://drive.google.com/drive/folders/1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'

In [ ]:
print("Getting file id...")
query = f"name = '{file_name}' and '{parent_id}' in parents and trashed = false"
response = (
    service.files()
    .list(q=query, spaces="drive", fields="files(id, name)")
    .execute()
)
print("processing response...")
for file in response.get("files", []):
    if file.get("name") == file_name:
        return file.get("id")
return None

In [59]:
for index, row in merged_df.iterrows():
    print(f"Processing {index+1}/{len(merged_df)}")
    json_file_name_only_name = row['jsonl_file_id'].replace(' ', '') + '.json'
    os.makedirs(DATA_DIR + 'FIXED5/', exist_ok=True)
    jsonl_filename = DATA_DIR + 'FIXED5/' + json_file_name_only_name
    with open(jsonl_filename, 'w') as jsonl_file:
        json.dump(row['jsonl_raw'], jsonl_file)

Processing 1/47
Processing 2/47
Processing 3/47
Processing 4/47
Processing 5/47
Processing 6/47
Processing 7/47
Processing 8/47
Processing 9/47
Processing 10/47
Processing 11/47
Processing 12/47
Processing 13/47
Processing 14/47
Processing 15/47
Processing 16/47
Processing 17/47
Processing 18/47
Processing 19/47
Processing 20/47
Processing 21/47
Processing 22/47
Processing 23/47
Processing 24/47
Processing 25/47
Processing 26/47
Processing 27/47
Processing 28/47
Processing 29/47
Processing 30/47
Processing 31/47
Processing 32/47
Processing 33/47
Processing 34/47
Processing 35/47
Processing 36/47
Processing 37/47
Processing 38/47
Processing 39/47
Processing 40/47
Processing 41/47
Processing 42/47
Processing 43/47
Processing 44/47
Processing 45/47
Processing 46/47
Processing 47/47


In [69]:
merged_df['colab_file_id'] = merged_df['task_link'].apply(lambda x: x.split('drive/')[1])
merged_df

,task_link,avg_score,min_score,issues,original_author_email,resolved_by_email,status,duration_mins,comments,jsonl_file_id,colab_link,messages,jsonl_raw,colab_file_id
0,https://colab.research.google.com/drive/1uc9gj...,3.833333,3,Overall - Completness: \n - Assistant's initia...,toh.y@turing.com,zain.v@turing.com,Done,10,None,1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur,https://colab.research.google.com/drive/1uc9gj...,"[{'role': 'User', 'content': 'I'm trying to co...",{'metadata': {'task_link': 'https://colab.rese...,1uc9gjNVbLismCaXtOYZJLOGidNH-EBJY
1,https://colab.research.google.com/drive/1A4jBO...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,toh.y@turing.com,zain.v@turing.com,Done,15,None,1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H,https://colab.research.google.com/drive/1A4jBO...,"[{'role': 'User', 'content': '```bash Tracebac...",{'metadata': {'task_link': 'https://colab.rese...,1A4jBOVVRiVjXRrSc6Khn5_nQKRj1wbmw
2,https://colab.research.google.com/drive/18C1mY...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,20,None,1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv,https://colab.research.google.com/drive/18C1mY...,"[{'role': 'User', 'content': 'Hello, I am runn...",{'metadata': {'task_link': 'https://colab.rese...,18C1mYX3acJ6Bm1HUHjjXZ86huUdDRb3r
3,https://colab.research.google.com/drive/1_Iypd...,3.833333,3,Overall - Completness: \n - The conversation l...,gautam.g@turing.com,khalid.s@turing.com,Done,45,None,17zNEPanui4_3NjvFe4184S0nNKRtt4HD,https://colab.research.google.com/drive/1_Iypd...,"[{'role': 'User', 'content': 'Could you demons...",{'metadata': {'task_link': 'https://colab.rese...,1_IypdQz9qthtLPX9bs2Xy--9VmUg75sx
4,https://colab.research.google.com/drive/1uA3D1...,3.833333,3,Overall - Completness: \n - Assistant's last r...,adil.m@turing.com,adil.m@turing.com,Done,30,None,1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH,https://colab.research.google.com/drive/1uA3D1...,"[{'role': 'User', 'content': 'Hi I want to pre...",{'metadata': {'task_link': 'https://colab.rese...,1uA3D1luYvAclXPMawVnJlRg9iElP08sK
5,https://colab.research.google.com/drive/1RqZj4...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,14,None,1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8,https://colab.research.google.com/drive/1RqZj4...,"[{'role': 'User', 'content': 'I'm working on a...",{'metadata': {'task_link': 'https://colab.rese...,1RqZj4QcRFD-zDf7fSNEU8qRjUaetOo1P
6,https://colab.research.google.com/drive/17LOxl...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,caram.v@turing.com,caram.v@turing.com,Done,30,None,1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb,https://colab.research.google.com/drive/17LOxl...,"[{'role': 'User', 'content': 'I have an e-comm...",{'metadata': {'task_link': 'https://colab.rese...,17LOxllYy5tR1DA1udlJ0bOrmT5vb546F
7,https://colab.research.google.com/drive/1PGIEJ...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,ishwar.b@turing.com,ishwar.b@turing.com,Done,45,None,1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg,https://colab.research.google.com/drive/1PGIEJ...,"[{'role': 'User', 'content': 'Hey, I am prepar...",{'metadata': {'task_link': 'https://colab.rese...,1PGIEJpxseizLKRdlgj0FrfFW190pT05Y
8,https://colab.research.google.com/drive/141lAs...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,andranik.g@turing.com,safi.u@turing.com,Done,25,None,1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc,https://colab.research.google.com/drive/141lAs...,"[{'role': 'User', 'content': 'I am working wit...",{'metadata': {'task_link': 'https://colab.rese...,141lAs8yJzWQ2R1nCnyILuWQUsWXO-eV6
9,https://colab.research.google.com/drive/1Vr2BH...,3.833333,3,Overall - Completness: \n - The assistant's re...,pawan.s@turing.com,zain.v@turing.com,Done,12,None,1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5,https://colab.research.google.com/drive/1Vr2BH...,"[{'role': 'User', 'content': 'Hi, I am getting...",{'metadata': {'task_link': 'https://colab.rese...,1Vr2BHasP5qvuBDnWX3Fo

In [70]:
from src.gdrive_api.folder_upload import upload_file
from src.gdrive_api.auth import build_service

for index, row in merged_df.iterrows():
    print(f"Processing {index+1}/{len(merged_df)}")
    json_file_name_only_name = row['colab_file_id'].replace(' ', '') + '.json'
    os.makedirs(DATA_DIR + 'FIXED5/', exist_ok=True)
    jsonl_filename = DATA_DIR + 'FIXED5/' + json_file_name_only_name
    with open(jsonl_filename, 'w') as jsonl_file:
        json.dump(row['jsonl_raw'], jsonl_file)
    upload_file(build_service(service_account_path), jsonl_filename, '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR', force_replace=True)

Processing 1/47
Getting file id...
processing response...
Replacing existing file '1uc9gjNVbLismCaXtOYZJLOGidNH-EBJY.json' with the new version.
File '1uc9gjNVbLismCaXtOYZJLOGidNH-EBJY.json' has been replaced.
Uploaded '1uc9gjNVbLismCaXtOYZJLOGidNH-EBJY.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 2/47
Getting file id...
processing response...
Replacing existing file '1A4jBOVVRiVjXRrSc6Khn5_nQKRj1wbmw.json' with the new version.
File '1A4jBOVVRiVjXRrSc6Khn5_nQKRj1wbmw.json' has been replaced.
Uploaded '1A4jBOVVRiVjXRrSc6Khn5_nQKRj1wbmw.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 3/47
Getting file id...
processing response...
Replacing existing file '18C1mYX3acJ6Bm1HUHjjXZ86huUdDRb3r.json' with the new version.
File '18C1mYX3acJ6Bm1HUHjjXZ86huUdDRb3r.json' has been replaced.
Uploaded '18C1mYX3acJ6Bm1HUHjjXZ86huUdDRb3r.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 4/47
Getting file id...
processing response...
Replacing ex

In [65]:
from src.gdrive_api.folder_upload import upload_file
from src.gdrive_api.auth import build_service

for index, row in merged_df.iterrows():
    print(f"Processing {index+1}/{len(merged_df)}")
    json_file_name_only_name = row['jsonl_file_id'].replace(' ', '') + '.json'
    os.makedirs(DATA_DIR + 'FIXED5/', exist_ok=True)
    jsonl_filename = DATA_DIR + 'FIXED5/' + json_file_name_only_name
    upload_file(build_service(service_account_path), jsonl_filename, '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR', force_replace=True)

Processing 1/47
Getting file id...
processing response...
Uploading new file '1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json'.
File '1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json' has been uploaded.
Uploaded '1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 2/47
Getting file id...
processing response...
Uploading new file '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json'.
File '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json' has been uploaded.
Uploaded '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 3/47
Getting file id...
processing response...
Uploading new file '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json'.
File '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json' has been uploaded.
Uploaded '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 4/47
Getting file id...
processing response...
Uploading new file '17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json'.
File '17zNEPanui4_3NjvFe4184S

KeyboardInterrupt: 

In [45]:
merged_df['jsonl_file_id'], merged_df['jsonl_raw']

,task_link,avg_score,min_score,issues,original_author_email,resolved_by_email,status,duration_mins,comments,jsonl_file_id,colab_link,messages,jsonl_raw
0,https://colab.research.google.com/drive/1uc9gj...,3.833333,3,Overall - Completness: \n - Assistant's initia...,toh.y@turing.com,zain.v@turing.com,Done,10,None,1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur,https://colab.research.google.com/drive/1uc9gj...,"[{'role': 'User', 'content': 'I'm trying to co...",{'metadata': {'task_link': 'https://colab.rese...
1,https://colab.research.google.com/drive/1A4jBO...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,toh.y@turing.com,zain.v@turing.com,Done,15,None,1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H,https://colab.research.google.com/drive/1A4jBO...,"[{'role': 'User', 'content': '```bash Tracebac...",{'metadata': {'task_link': 'https://colab.rese...
2,https://colab.research.google.com/drive/18C1mY...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,20,None,1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv,https://colab.research.google.com/drive/18C1mY...,"[{'role': 'User', 'content': 'Hello, I am runn...",{'metadata': {'task_link': 'https://colab.rese...
3,https://colab.research.google.com/drive/1_Iypd...,3.833333,3,Overall - Completness: \n - The conversation l...,gautam.g@turing.com,khalid.s@turing.com,Done,45,None,17zNEPanui4_3NjvFe4184S0nNKRtt4HD,https://colab.research.google.com/drive/1_Iypd...,"[{'role': 'User', 'content': 'Could you demons...",{'metadata': {'task_link': 'https://colab.rese...
4,https://colab.research.google.com/drive/1uA3D1...,3.833333,3,Overall - Completness: \n - Assistant's last r...,adil.m@turing.com,adil.m@turing.com,Done,30,None,1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH,https://colab.research.google.com/drive/1uA3D1...,"[{'role': 'User', 'content': 'Hi I want to pre...",{'metadata': {'task_link': 'https://colab.rese...
5,https://colab.research.google.com/drive/1RqZj4...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,sudharchith.s@turing.com,zain.v@turing.com,Done,14,None,1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8,https://colab.research.google.com/drive/1RqZj4...,"[{'role': 'User', 'content': 'I'm working on a...",{'metadata': {'task_link': 'https://colab.rese...
6,https://colab.research.google.com/drive/17LOxl...,3.833333,2,Overall - Completness: \n \n \n User - Natural...,caram.v@turing.com,caram.v@turing.com,Done,30,None,1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb,https://colab.research.google.com/drive/17LOxl...,"[{'role': 'User', 'content': 'I have an e-comm...",{'metadata': {'task_link': 'https://colab.rese...
7,https://colab.research.google.com/drive/1PGIEJ...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,ishwar.b@turing.com,ishwar.b@turing.com,Done,45,None,1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg,https://colab.research.google.com/drive/1PGIEJ...,"[{'role': 'User', 'content': 'Hey, I am prepar...",{'metadata': {'task_link': 'https://colab.rese...
8,https://colab.research.google.com/drive/141lAs...,3.833333,3,Overall - Completness: \n \n \n User - Natural...,andranik.g@turing.com,safi.u@turing.com,Done,25,None,1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc,https://colab.research.google.com/drive/141lAs...,"[{'role': 'User', 'content': 'I am working wit...",{'metadata': {'task_link': 'https://colab.rese...
9,https://colab.research.google.com/drive/1Vr2BH...,3.833333,3,Overall - Completness: \n - The assistant's re...,pawan.s@turing.com,zain.v@turing.com,Done,12,None,1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5,https://colab.research.google.com/drive/1Vr2BH...,"[{'role': 'User', 'content': 'Hi, I am getting...",{'metadata': {'task_link': 'https://colab.rese...


In [75]:
log = """
Processing 1/47
Getting file id...
processing response...
Uploading new file '1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json'.
File '1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json' has been uploaded.
Uploaded '1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 2/47
Getting file id...
processing response...
Uploading new file '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json'.
File '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json' has been uploaded.
Uploaded '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 3/47
Getting file id...
processing response...
Uploading new file '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json'.
File '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json' has been uploaded.
Uploaded '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 4/47
Getting file id...
processing response...
Uploading new file '17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json'.
File '17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json' has been uploaded.
Uploaded '17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 5/47
Getting file id...
processing response...
Uploading new file '1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH.json'.
File '1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH.json' has been uploaded.
Uploaded '1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 6/47
Getting file id...
processing response...
Uploading new file '1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8.json'.
File '1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8.json' has been uploaded.
Uploaded '1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 7/47
Getting file id...
processing response...
Uploading new file '1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb.json'.
File '1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb.json' has been uploaded.
Uploaded '1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 8/47
Getting file id...
processing response...
Uploading new file '1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg.json'.
File '1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg.json' has been uploaded.
Uploaded '1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 9/47
Getting file id...
processing response...
Uploading new file '1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc.json'.
File '1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc.json' has been uploaded.
Uploaded '1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 10/47
Getting file id...
processing response...
Uploading new file '1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5.json'.
File '1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5.json' has been uploaded.
Uploaded '1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 11/47
Getting file id...
processing response...
Uploading new file '19ayQmU_oCA7OV7PZQWsQMNAFlYLu4FYR.json'.
File '19ayQmU_oCA7OV7PZQWsQMNAFlYLu4FYR.json' has been uploaded.
Uploaded '19ayQmU_oCA7OV7PZQWsQMNAFlYLu4FYR.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 12/47
Getting file id...
processing response...
Uploading new file '1XZA-5VAgKVzgTfKVkAOzqFCT6TubdSL8.json'.
File '1XZA-5VAgKVzgTfKVkAOzqFCT6TubdSL8.json' has been uploaded.
Uploaded '1XZA-5VAgKVzgTfKVkAOzqFCT6TubdSL8.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 13/47
Getting file id...
processing response...
Uploading new file '1aIcZpqvtJ3cx3uvKkJZGCl0GE9iTZdUg.json'.
File '1aIcZpqvtJ3cx3uvKkJZGCl0GE9iTZdUg.json' has been uploaded.
Uploaded '1aIcZpqvtJ3cx3uvKkJZGCl0GE9iTZdUg.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 14/47
Getting file id...
processing response...
Uploading new file '13Lzn_K-x5mU-cOZJ0VFWwWHNRyjXFJvL.json'.
File '13Lzn_K-x5mU-cOZJ0VFWwWHNRyjXFJvL.json' has been uploaded.
Uploaded '13Lzn_K-x5mU-cOZJ0VFWwWHNRyjXFJvL.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 15/47
Getting file id...
processing response...
Uploading new file '1rcice9l8_TbK2TxUA-etr12V-pIB4Mgq.json'.
File '1rcice9l8_TbK2TxUA-etr12V-pIB4Mgq.json' has been uploaded.
Uploaded '1rcice9l8_TbK2TxUA-etr12V-pIB4Mgq.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 16/47
Getting file id...
processing response...
Uploading new file '1DIFoxE5xUUl8W9y-HNiahMMdlsP_VOHM.json'.
File '1DIFoxE5xUUl8W9y-HNiahMMdlsP_VOHM.json' has been uploaded.
Uploaded '1DIFoxE5xUUl8W9y-HNiahMMdlsP_VOHM.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 17/47
Getting file id...
processing response...
Uploading new file '1o8pxUdQjYcCV4_0-Aeqxqqx_tRArjVm4.json'.
File '1o8pxUdQjYcCV4_0-Aeqxqqx_tRArjVm4.json' has been uploaded.
Uploaded '1o8pxUdQjYcCV4_0-Aeqxqqx_tRArjVm4.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 18/47
Getting file id...
processing response...
Uploading new file '1j0EIJbAM1ChKmzgJgu5aGU1b5LTfZtgl.json'.
File '1j0EIJbAM1ChKmzgJgu5aGU1b5LTfZtgl.json' has been uploaded.
Uploaded '1j0EIJbAM1ChKmzgJgu5aGU1b5LTfZtgl.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 19/47
Getting file id...
processing response...
Uploading new file '13VywbfgFWYAd_3ckWmFgVaMeRAahKs_v.json'.
File '13VywbfgFWYAd_3ckWmFgVaMeRAahKs_v.json' has been uploaded.
Uploaded '13VywbfgFWYAd_3ckWmFgVaMeRAahKs_v.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 20/47
Getting file id...
processing response...
Uploading new file '1gy5or8jSc7bUpXLWpZnHlPnK62G3xGqJ.json'.
File '1gy5or8jSc7bUpXLWpZnHlPnK62G3xGqJ.json' has been uploaded.
Uploaded '1gy5or8jSc7bUpXLWpZnHlPnK62G3xGqJ.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 21/47
Getting file id...
processing response...
Uploading new file '1erYgkIaMRctwU7jHKsSGWeri2jb1Mfjv.json'.
File '1erYgkIaMRctwU7jHKsSGWeri2jb1Mfjv.json' has been uploaded.
Uploaded '1erYgkIaMRctwU7jHKsSGWeri2jb1Mfjv.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 22/47
Getting file id...
processing response...
Uploading new file '17ew_SjJFdEA3ZJB_q-8T_NmEc8gCGUf3.json'.
File '17ew_SjJFdEA3ZJB_q-8T_NmEc8gCGUf3.json' has been uploaded.
Uploaded '17ew_SjJFdEA3ZJB_q-8T_NmEc8gCGUf3.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 23/47
Getting file id...
processing response...
Uploading new file '1AW2WCHF66064dBNHyAjNV898RH8djZxY.json'.
File '1AW2WCHF66064dBNHyAjNV898RH8djZxY.json' has been uploaded.
Uploaded '1AW2WCHF66064dBNHyAjNV898RH8djZxY.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 24/47
Getting file id...
processing response...
Uploading new file '1CAw4gAHPBewjn-513Oh8EHUUzhJnp57u.json'.
File '1CAw4gAHPBewjn-513Oh8EHUUzhJnp57u.json' has been uploaded.
Uploaded '1CAw4gAHPBewjn-513Oh8EHUUzhJnp57u.json' to folder ID '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'.
Processing 25/47
Getting file id...
processing response...
Uploading new file '1DbVKF30oMVp7VPaiIF_381hl6-ci_Vgt.json'.
"""
import re
filenames = re.findall(r"Uploading new file '([\w\d_\-\.]+)'", log)

filenames


['1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json',
 '1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json',
 '1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json',
 '17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json',
 '1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH.json',
 '1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8.json',
 '1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb.json',
 '1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg.json',
 '1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc.json',
 '1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5.json',
 '19ayQmU_oCA7OV7PZQWsQMNAFlYLu4FYR.json',
 '1XZA-5VAgKVzgTfKVkAOzqFCT6TubdSL8.json',
 '1aIcZpqvtJ3cx3uvKkJZGCl0GE9iTZdUg.json',
 '13Lzn_K-x5mU-cOZJ0VFWwWHNRyjXFJvL.json',
 '1rcice9l8_TbK2TxUA-etr12V-pIB4Mgq.json',
 '1DIFoxE5xUUl8W9y-HNiahMMdlsP_VOHM.json',
 '1o8pxUdQjYcCV4_0-Aeqxqqx_tRArjVm4.json',
 '1j0EIJbAM1ChKmzgJgu5aGU1b5LTfZtgl.json',
 '13VywbfgFWYAd_3ckWmFgVaMeRAahKs_v.json',
 '1gy5or8jSc7bUpXLWpZnHlPnK62G3xGqJ.json',
 '1erYgkIaMRctwU7jHKsSGWeri2jb1Mfjv.json',
 '17ew_SjJFdEA3ZJB_q-8T_NmEc8gCGUf3.json',
 '1AW2WCHF66064dBNHyAjNV898RH8djZxY.json',
 '1CAw4gAHP

In [79]:
len(filenames)

25

In [78]:
for search_term in filenames:
    print(f"https://drive.google.com/drive/search?q={search_term}%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR")

https://drive.google.com/drive/search?q=1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com/drive/search?q=1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg.json%20parent:1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR
https://drive.google.com

In [84]:
gdrive_service = build_service(service_account_path)

def delete_file(service, file_name, parent_id):
    try:
        # Search for the file by name and parent ID to get its file ID
        response = service.files().list(q=f"name = '{file_name}' and '{parent_id}' in parents").execute()
        assert len(response.get('files', [])) == 1
        for file in response.get('files', []):
            # Call the Drive v3 API to delete the file
            service.files().delete(fileId=file['id']).execute()
            print(f"Deleted file with name: {file_name} and id: {file['id']}")
    except Exception as error:
        print(f"An error occurred: {error}")

parent_id = '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'
for filename in filenames:
    delete_file(gdrive_service, filename, parent_id)


Deleted file with name: 1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur.json and id: 1vsl9N9kNOKjSvx-yDr-Z9YR02tpvYNpy
Deleted file with name: 1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H.json and id: 1ZlpY-Ro92QnNakCSfcwKhq0aRsI1F9rS
Deleted file with name: 1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv.json and id: 1A8jzG2uCe0Pn4xi7OedKlpAMwx7p2I0X
Deleted file with name: 17zNEPanui4_3NjvFe4184S0nNKRtt4HD.json and id: 1G-_lhmyUjbLQlqFPowPAfQk1T73uPtPA
Deleted file with name: 1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH.json and id: 1q6TamjnDBmsOzJxOVIBAqElxglY2_hiL
Deleted file with name: 1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8.json and id: 1R-FqQ0Q1fzkHyI9IowD0G0VB6_Am-BVQ
Deleted file with name: 1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb.json and id: 1_PIR2_2qfiOpJKnXJh-6rA_6BuQFdWby
Deleted file with name: 1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg.json and id: 13ibVEdkv6zOde6A3ShHvqql2Aj8Bt5-Z
Deleted file with name: 1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc.json and id: 1uHDeOJ9depbk5DnPMc23gMHm0kLH1GJ4
Deleted file with name: 1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy

In [83]:
merged_df['jsonl_file_id']

0     1SPOs9PfhcEFPE7khR-pHErf6CVVlW1Ur
1     1fXD6WK7Xm0YsbfrE_dXVqPeWD-EpEW7H
2     1dYt_IxdCOjB5LaNDeL5M4FWI21smIgAv
3     17zNEPanui4_3NjvFe4184S0nNKRtt4HD
4     1fFVYfCF8wYBGE_spojlRx9pSq-UmjPBH
5     1O0hZVjovDLwTuMBstUthXptVcsDHzIZ8
6     1-UP5vhiUnHh0SDaP65yT7cIWst_sFnfb
7     1CebcC4lixsqy3W0FtaEkEfjZ-0M-zcNg
8     1mDnjiklmV7w8tC5Flu472vNP_d-K4Ygc
9     1FUE_JOMLS4wKXEvGzR2AeNuwopEXiy_5
10    19ayQmU_oCA7OV7PZQWsQMNAFlYLu4FYR
11    1XZA-5VAgKVzgTfKVkAOzqFCT6TubdSL8
12    1aIcZpqvtJ3cx3uvKkJZGCl0GE9iTZdUg
13    13Lzn_K-x5mU-cOZJ0VFWwWHNRyjXFJvL
14    1rcice9l8_TbK2TxUA-etr12V-pIB4Mgq
15    1DIFoxE5xUUl8W9y-HNiahMMdlsP_VOHM
16    1o8pxUdQjYcCV4_0-Aeqxqqx_tRArjVm4
17    1j0EIJbAM1ChKmzgJgu5aGU1b5LTfZtgl
18    13VywbfgFWYAd_3ckWmFgVaMeRAahKs_v
19    1gy5or8jSc7bUpXLWpZnHlPnK62G3xGqJ
20    1erYgkIaMRctwU7jHKsSGWeri2jb1Mfjv
21    17ew_SjJFdEA3ZJB_q-8T_NmEc8gCGUf3
22    1AW2WCHF66064dBNHyAjNV898RH8djZxY
23    1CAw4gAHPBewjn-513Oh8EHUUzhJnp57u
24    1DbVKF30oMVp7VPaiIF_381hl6-ci_Vgt


delivered_df['task_link]